In [7]:
import numpy as np, pandas as pd # CPU LIBRARIES
import matplotlib.pyplot as plt, gc
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
LIMIT = 8
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    except RuntimeError as e:
        print(e)
print('We will restrict TensorFlow to max %iGB GPU RAM'%LIMIT)
print('then RAPIDS can use %iGB GPU RAM'%(16-LIMIT))



We will restrict TensorFlow to max 8GB GPU RAM
then RAPIDS can use 8GB GPU RAM


In [9]:
PATH_TO_CUSTOMER_HASHES = '../input/amex-data-files/'

# AFTER PROCESSING DATA ONCE, UPLOAD TO KAGGLE DATASET
# THEN SET VARIABLE BELOW TO FALSE
# AND ATTACH DATASET TO NOTEBOOK AND PUT PATH TO DATASET BELOW
PROCESS_DATA = True
PATH_TO_DATA = './data/'
PATH_TO_DATA = '../input/amex-data-for-transformers-and-rnns/data/'

# AFTER TRAINING MODEL, UPLOAD TO KAGGLE DATASET
# THEN SET VARIABLE BELOW TO FALSE
# AND ATTACH DATASET TO NOTEBOOK AND PUT PATH TO DATASET BELOW
TRAIN_MODEL = True
PATH_TO_MODEL = './model/'
PATH_TO_MODEL = '../input/amex-data-for-transformers-and-rnns/model/'

INFER_TEST = True


In [ ]:

import numpy as np, pandas as pd # CPU LIBRARIES
import matplotlib.pyplot as plt, gc

if PROCESS_DATA:
    # LOAD TARGETS
    targets = pd.read_csv('train_data.csv')
    targets['customer_ID'] = targets['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
    print(f'There are {targets.shape[0]} train targets')

    # GET TRAIN COLUMN NAMES
    train = pd.read_csv('train_data.csv', nrows=1)
    T_COLS = train.columns
    print(f'There are {len(T_COLS)} train dataframe columns')

    # GET TRAIN CUSTOMER NAMES (use pandas to avoid memory error)
    if PATH_TO_CUSTOMER_HASHES:
        train = pd.read_parquet(f'{PATH_TO_CUSTOMER_HASHES}train_customer_hashes.pqt')
    else:
        train = pd.read_csv('train_data.csv', usecols=['customer_ID'])
        train['customer_ID'] = train['customer_ID'].apply(lambda x: int(x[-16:],16) ).astype('int64')
    customers = train.drop_duplicates().sort_index().values.flatten()
    print(f'There are {len(customers)} unique customers in train.')

In [ ]:
def get_rows(customers, train, NUM_FILES = 10, verbose = ''):
    chunk = len(customers)//NUM_FILES
    if verbose != '':
        print(f'We will split {verbose} data into {NUM_FILES} separate files.')
        print(f'There will be {chunk} customers in each file (except the last file).')
        print('Below are number of rows in each file:')
    rows = []

    for k in range(NUM_FILES):
        if k==NUM_FILES-1: cc = customers[k*chunk:]
        else: cc = customers[k*chunk:(k+1)*chunk]
        s = train.loc[train.customer_ID.isin(cc)].shape[0]
        rows.append(s)
    if verbose != '': print( rows )
    return rows
    rows = get_rows(customers, train, NUM_FILES = NUM_FILES, verbose = 'train')

In [2]:
print(1)

1


In [3]:
# def feature_engineer(train, PAD_CUSTOMER_TO_13_ROWS = True, targets = None):
#
#     # REDUCE STRING COLUMNS
#     # from 64 bytes to 8 bytes, and 10 bytes to 3 bytes respectively
#     train['customer_ID'] = train['customer_ID'].str[-16:].str.hex_to_int().astype('int64')
#     train.S_2 = cudf.to_datetime( train.S_2 )
#     train['year'] = (train.S_2.dt.year-2000).astype('int8')
#     train['month'] = (train.S_2.dt.month).astype('int8')
#     train['day'] = (train.S_2.dt.day).astype('int8')
#     del train['S_2']
#
#     # LABEL ENCODE CAT COLUMNS (and reduce to 1 byte)
#     # with 0: padding, 1: nan, 2,3,4,etc: values
#     d_63_map = {'CL':2, 'CO':3, 'CR':4, 'XL':5, 'XM':6, 'XZ':7}
#     train['D_63'] = train.D_63.map(d_63_map).fillna(1).astype('int8')
#
#     d_64_map = {'-1':2,'O':3, 'R':4, 'U':5}
#     train['D_64'] = train.D_64.map(d_64_map).fillna(1).astype('int8')
#
#     CATS = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_66', 'D_68']
#     OFFSETS = [2,1,2,2,3,2,3,2,2] #2 minus minimal value in full train csv
#     # then 0 will be padding, 1 will be NAN, 2,3,4,etc will be values
#     for c,s in zip(CATS,OFFSETS):
#         train[c] = train[c] + s
#         train[c] = train[c].fillna(1).astype('int8')
#     CATS += ['D_63','D_64']
#
#     # ADD NEW FEATURES HERE
#     # EXAMPLE: train['feature_189'] = etc etc etc
#     # EXAMPLE: train['feature_190'] = etc etc etc
#     # IF CATEGORICAL, THEN ADD TO CATS WITH: CATS += ['feaure_190'] etc etc etc
#
#     # REDUCE MEMORY DTYPE
#     SKIP = ['customer_ID','year','month','day']
#     for c in train.columns:
#         if c in SKIP: continue
#         if str( train[c].dtype )=='int64':
#             train[c] = train[c].astype('int32')
#         if str( train[c].dtype )=='float64':
#             train[c] = train[c].astype('float32')
#
#     # PAD ROWS SO EACH CUSTOMER HAS 13 ROWS
#     if PAD_CUSTOMER_TO_13_ROWS:
#         tmp = train[['customer_ID']].groupby('customer_ID').customer_ID.agg('count')
#         more = cupy.array([],dtype='int64')
#         for j in range(1,13):
#             i = tmp.loc[tmp==j].index.values
#             more = cupy.concatenate([more,cupy.repeat(i,13-j)])
#         df = train.iloc[:len(more)].copy().fillna(0)
#         df = df * 0 - 1 #pad numerical columns with -1
#         df[CATS] = (df[CATS] * 0).astype('int8') #pad categorical columns with 0
#         df['customer_ID'] = more
#         train = cudf.concat([train,df],axis=0,ignore_index=True)
#
#     # ADD TARGETS (and reduce to 1 byte)
#     if targets is not None:
#         train = train.merge(targets,on='customer_ID',how='left')
#         train.target = train.target.astype('int8')
#
#     # FILL NAN
#     train = train.fillna(-0.5) #this applies to numerical columns
#
#     # SORT BY CUSTOMER THEN DATE
#     train = train.sort_values(['customer_ID','year','month','day']).reset_index(drop=True)
#     train = train.drop(['year','month','day'],axis=1)
#
#     # REARRANGE COLUMNS WITH 11 CATS FIRST
#     COLS = list(train.columns[1:])
#     COLS = ['customer_ID'] + CATS + [c for c in COLS if c not in CATS]
#     train = train[COLS]
#
#     return train